In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/base_file.csv')

In [5]:
x = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.lon,df.lat))

In [6]:
def check_country(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_region = gdf['ADM0_PCODE'].iloc[count]
            return new_region
        else:
            count +=1

In [7]:
def check_region(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_region = gdf['ADM1_PCODE'].iloc[count]
            return new_region
        else:
            count +=1

In [8]:
def check_zone(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_zone = gdf['ADM2_PCODE'].iloc[count]
            return new_zone
        else:
            count +=1

In [9]:
def check_woreda(foo, gdf):
    count = 0
    while count < len(gdf):
        test = foo['geometry'].within(gdf['geometry'].iloc[count])
        if test == True:
            new_woreda = gdf['ADM3_PCODE'].iloc[count]
            return new_woreda
        else:
            count +=1

In [11]:
# read in the shape files in geojson format. 
gdf_country = gpd.read_file('eth_shape_files/json/eth_admin0v2.json')
gdf_region = gpd.read_file('eth_shape_files/json//eth_admin1v2.json')
gdf_zone = gpd.read_file('eth_shape_files/json//eth_admin2v2.json')
gdf_woreda = gpd.read_file('eth_shape_files/json//eth_admin3v2.json')

In [12]:
# Create placeholder columns
x['ADM0_PCODE'] = ''
x['ADM1_PCODE'] = ''
x['ADM2_PCODE'] = ''
x['ADM3_PCODE'] = ''

In [13]:
# Run function on all GPS coordinates in dataset for country

i = 0
count = 0
while i < len(x):
        bb = check_country(x.iloc[i], gdf_country)
        x.loc[i, 'ADM0_PCODE'] = bb
        i +=1

In [16]:
# Run function on all GPS coordinates in dataset for region

i = 0
while i < len(x):
        bb = check_region(x.iloc[i], gdf_region)
        x.loc[i, 'ADM1_PCODE'] = bb
        i +=1

In [17]:
# Run function on all GPS coordinates in dataset for zone

i = 0
while i < len(x):
        bb = check_zone(x.iloc[i], gdf_zone)
        x.loc[i, 'ADM2_PCODE'] = bb
        i +=1

In [18]:
# Run function on all GPS coordinates in dataset for woreda

i = 0
while i < len(x):
        bb = check_woreda(x.iloc[i], gdf_woreda)
        x.loc[i, 'ADM3_PCODE'] = bb
        i +=1

In [19]:
# Review the dataset again and observe the differences between the administrative boundaries. 
# The original region, zone and woreda can be deleted at this stage and replaced with the corrected admin names.
# FYI Woreda = District in Ethiopia.

x.head()

region            zone        woreda         school_name   admin_code  \
0  Amhara  Misrak  Gojjam  Hulet_Ej_Ene              Tekort  S0306020712   
1  Amhara  Misrak  Gojjam  Hulet_Ej_Ene  Boda Kidanemihiret  S0306021792   
2  Amhara  Misrak  Gojjam  Hulet_Ej_Ene              Meklit  S0306020332   
3  Amhara  Misrak  Gojjam         Sedie             Grarema  S0306020292   
4  Amhara  Misrak  Gojjam         Sedie                Jema  S0306025422   

                    geometry        lat       lon gr_offer  urban_rural  ...  \
0  POINT (37.98730 11.09990)  11.099900  37.98730    G.1-4            1  ...   
1  POINT (37.69205 11.05947)  11.059470  37.69205    G.1-4            1  ...   
2  POINT (37.79732 11.07784)  11.077843  37.79732    G.1-8            1  ...   
3  POINT (37.91040 10.74920)  10.749200  37.91040    G.1-8            1  ...   
4  POINT (37.80780 10.88000)  10.880000  37.80780    G.1-8            1  ...   

   grade11_12  grade11_12_f  grade11_12_m  water_y_n  has_separate_toilet  \
0           0             0             0        1.0                   No   
1           0             0             0        1.0                   No   
2           0             0             0        1.0                   No   
3           0             0             0        1.0                  Yes   
4           0             0             0        NaN                  Yes   

   has_toilet  ADM0_PCODE  ADM1_PCODE  ADM2_PCODE  ADM3_PCODE  
0         Yes          ET        ET03      ET0306    ET030602  
1         Yes          ET        ET03      ET0306    ET030602  
2         Yes          ET        ET03      ET0306    ET030602  
3         Yes          ET        ET03      ET0306    ET030602  
4         Yes          ET        ET03      ET0306    ET030602  

[5 rows x 29 columns]

In [20]:
# Merge datasets basdd on the administrative codes. In this case, it is the PCODE

x= x.merge(gdf_woreda[['ADM0_PCODE', 'ADM0_EN', 'ADM1_PCODE', 'ADM1_EN', 'ADM2_PCODE', 'ADM2_EN', 'ADM3_PCODE', 'ADM3_EN']], 
             how='inner', on=['ADM0_PCODE','ADM1_PCODE', 'ADM2_PCODE', 'ADM3_PCODE'])

In [22]:
# Write out the CSV 
x.to_csv('data/gis_correct_admin.csv', index=True, encoding = 'utf-8')